In [17]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from tensorflow.keras.preprocessing import sequence

import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
train = pd.read_csv('train.csv')
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [6]:
species_ = train['species'].unique()

species_dict = {}
for i in range(len(species_)):
    species_dict['{}'.format(species_[i])] = i+1

print(species_dict)

train['species'] = train['species'].map(species_dict)
train.head()

{'T': 1, 'P': 2, 'Z': 3}


,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,1,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,1,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,1,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,1,Ability,(1360) - TrainSCV
4,0,1,0.14,0,1,Camera,"at (142.99609375, 24.50390625)"


In [5]:
1


1

In [7]:
events = train['event'].unique()

dict_events = {}
for i in range(len(events)):
    dict_events['{}'.format(events[i])] = i+1
    
print(dict_events)
train['event']=train['event'].map(dict_events)
train.head()

{'Camera': 1, 'Selection': 2, 'Ability': 3, 'Right Click': 4, 'SetControlGroup': 5, 'GetControlGroup': 6, 'AddToControlGroup': 7, 'ControlGroup': 8}


,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,1,1,"at (145.25, 21.5078125)"
1,0,1,0.00,1,1,1,"at (22.75, 147.0078125)"
2,0,1,0.02,0,1,2,['OrbitalCommand [3080001]']
3,0,1,0.02,0,1,3,(1360) - TrainSCV
4,0,1,0.14,0,1,1,"at (142.99609375, 24.50390625)"


In [8]:
target = pd.DataFrame()

target['game_id'] = train['game_id']
target['winner'] = train['winner']
target = target.drop_duplicates(['game_id'], keep='first')
target = target.reset_index(drop=True)
target.head()

,game_id,winner
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0


In [9]:
ids = train['game_id'].unique()

train.drop(['event_contents','time','winner'], axis = 1, inplace=True)
train.head()

,game_id,player,species,event
0,0,0,1,1
1,0,1,1,1
2,0,0,1,2
3,0,0,1,3
4,0,0,1,1


In [11]:
idx = train['game_id']
idx

0               0
1               0
2               0
3               0
4               0
            ...  
67091771    38871
67091772    38871
67091773    38871
67091774    38871
67091775    38871
Name: game_id, Length: 67091776, dtype: int64

In [12]:
normalized_train=(train-train.min())/(train.max()-train.min())
normalized_train['game_id'] = idx

In [18]:
normalized_train.head()

,game_id,player,species,event
0,0,0.0,0.0,0.000000
1,0,1.0,0.0,0.000000
2,0,0.0,0.0,0.142857
3,0,0.0,0.0,0.285714
4,0,0.0,0.0,0.000000


In [19]:
sequences = list()

for i in tqdm(ids):
    df = normalized_train[normalized_train['game_id']==ids[i]]
    df.drop(['game_id'],axis=1, inplace=True)
    
    sequences.append(df.values)
    
    

100%|██████████| 38872/38872 [42:04<00:00, 15.40it/s]


In [20]:
seq_prime = sequence.pad_sequences(sequences,padding='post', value=0)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(seq_prime, target['winner'], test_size=0.1)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM

In [27]:
input_shape=np.shape(seq_prime[0])

In [28]:
model = Sequential()
model.add(LSTM(100, input_shape=(input_shape),return_sequences = True))
model.add(LSTM(20, return_sequences = True))
model.add(LSTM(20, return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5164, 100)         41600     
_________________________________________________________________
lstm_1 (LSTM)                (None, 5164, 20)          9680      
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 56,761
Trainable params: 56,761
Non-trainable params: 0
____________________________________________________

In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [31]:
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1, validation_split=0.2)

Train on 27987 samples, validate on 6997 samples
Epoch 1/20
27987/27987 [==============================] - 681s 24ms/sample - loss: 7.5693 - accuracy: 0.5036 - val_loss: 7.7281 - val_accuracy: 0.4932
Epoch 2/20
27987/27987 [==============================] - 676s 24ms/sample - loss: 7.5693 - accuracy: 0.5036 - val_loss: 7.7281 - val_accuracy: 0.4932
Epoch 3/20
27987/27987 [==============================] - 676s 24ms/sample - loss: 7.5693 - accuracy: 0.5036 - val_loss: 7.7281 - val_accuracy: 0.4932
Epoch 4/20
27987/27987 [==============================] - 676s 24ms/sample - loss: 7.5693 - accuracy: 0.5036 - val_loss: 7.7281 - val_accuracy: 0.4932
Epoch 5/20
27987/27987 [==============================] - 678s 24ms/sample - loss: 7.5693 - accuracy: 0.5036 - val_loss: 7.7281 - val_accuracy: 0.4932
Epoch 6/20
27987/27987 [==============================] - 676s 24ms/sample - loss: 7.5693 - accuracy: 0.5036 - val_loss: 7.7281 - val_accuracy: 0.4932
Epoch 7/20
27987/27987 [=====================

KeyboardInterrupt: 

In [ ]:
1